In [1]:
!pip install scipy==1.7.3
!pip install numpy==1.23.5
!pip install pandas==1.5.3
!pip install lightgbm==3.3.2
!pip install catboost==1.2
!pip install xgboost==1.7.6
!pip install sklearn==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 84.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.1
    Uninstalling scipy-1.11.1:
      Successfully uninstalled scipy-1.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
chex 0.1.81 requires numpy>=1.25.0, but you have numpy 1.22.4 which is incompatible.
momepy 0.6.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pointpats 2.3.0 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.
py

In [2]:
import scipy
import numpy
import pandas
import lightgbm
import catboost
import xgboost
import sklearn
print(scipy.__version__)
print(numpy.__version__)
print(pandas.__version__)
print(lightgbm.__version__)
print(catboost.__version__)
print(xgboost.__version__)
print(sklearn.__version__)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


1.7.3
1.23.5
1.5.3
3.3.2
1.2
1.7.6
1.2.2


In [3]:
import os
import warnings
import statistics
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import lightgbm as lgbm
from scipy import stats
from scipy.stats import hmean,gmean
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GroupKFold
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")

In [4]:
version_nb = 'v6_kaggle'

In [5]:
df_meta_train = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/train.csv',parse_dates=['endTime'])
df_meta_test = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/test.csv',parse_dates=['endTime'])
df_ss = pd.read_csv('/kaggle/input/fia-zindi-kp-data-creation/sample_submission.csv')
df_meta_test.sort_values(['ID','endTime'],ascending=True,ignore_index=True,inplace=True)
df_meta_train.sort_values(['ID','endTime'],ascending=True,ignore_index=True,inplace=True)
print(df_meta_train.shape, df_meta_test.shape, df_ss.shape)
display(df_meta_train.head(1))
display(df_meta_test.head(1))
display(df_ss.head(1))

(908922, 12) (175150, 12) (1932, 2)


,ID,NE ID,endTime,access_success_rate,resource_utilition_rate,TA,bler,cqi,mcs,data_rate,fault_duration,relation
0,B0001-12_1,B0001-12,2023-02-09 01:00:00,99.519744,29.417,0.636289,10.12688,8.640631,12.523579,32.209893,0.0,0.0


,ID,NE ID,endTime,access_success_rate,resource_utilition_rate,TA,bler,cqi,mcs,data_rate,fault_duration,relation
0,B0017-25_24,B0017-25,2023-02-20 06:00:00,99.714558,8.678,3.71541,7.653169,5.964988,5.403557,6.815789,0.0,0.0


,ID,data_rate_t+1_trend
0,B0017-25_24,0


In [6]:
df_meta_train_features = df_meta_train[df_meta_train['fault_duration']==0].reset_index(drop=True)
df_meta_test_features = df_meta_test[df_meta_test['fault_duration']==0].reset_index(drop=True)
print(df_meta_train_features.shape,df_meta_test_features.shape)
df_train = df_meta_train[df_meta_train['fault_duration']>0].reset_index(drop=True)
df1 = df_meta_train_features.groupby(['ID'])[['data_rate']].last().add_suffix("_t-1").reset_index()
df_train = df_train.merge(df1,on=['ID'],how='left')
df_train['data_rate_t+1_trend'] = np.where(df_train['data_rate']<df_train['data_rate_t-1'],1,0)


# df_train_first = df_train.groupby(['ID']).first().reset_index()
# df_train_after_first = df_train.groupby(["ID"]).apply(lambda group: group.iloc[1:]).reset_index(drop=True)
for col in ['access_success_rate','resource_utilition_rate', 'TA', 'bler', 'cqi', 'mcs']:
    del df_train[col]
# df_train = pd.concat([df_train_first,df_train_after_first], ignore_index=True)

print(df_train.groupby(['ID']).first()['data_rate_t+1_trend'].value_counts(normalize=True))
print(df_train['data_rate_t+1_trend'].value_counts(normalize=True))
df_train.rename(columns={'data_rate':'TARGET_data_rate','data_rate_t+1_trend':'TARGET_data_rate_t+1_trend'},inplace=True)
df_train = df_train[['ID', 'NE ID', 'endTime',  'fault_duration','relation', 'data_rate_t-1',
                     'TARGET_data_rate', 'TARGET_data_rate_t+1_trend']].copy()
display(df_train)
df_test = df_meta_test[df_meta_test['fault_duration']>0].reset_index(drop=True)
df1 = df_meta_test_features.groupby(['ID'])[['data_rate']].last().add_suffix("_t-1").reset_index()
df_test = df_test.merge(df1,on=['ID'],how='left')
for col in ['access_success_rate','resource_utilition_rate', 'TA', 'bler', 'cqi', 'mcs','data_rate']:
    del df_test[col]

display(df_test)
df_meta_train_features['data'] = 'no_fault'
df_meta_test_features['data'] = 'no_fault'
df_train['data'] = 'train_fault'
df_test['data'] = 'test_fault'
df_combined = pd.concat([df_meta_train_features,df_train,df_meta_test_features,df_test]).reset_index(drop=True)

(813138, 12) (173218, 12)
0    0.508407
1    0.491593
Name: data_rate_t+1_trend, dtype: float64
1    0.526936
0    0.473064
Name: data_rate_t+1_trend, dtype: float64


,ID,NE ID,endTime,fault_duration,relation,data_rate_t-1,TARGET_data_rate,TARGET_data_rate_t+1_trend
0,B0001-12_1,B0001-12,2023-02-20 13:00:00,91.0,0.942467,23.253083,23.910174,0
1,B0001-13_1,B0001-13,2023-02-20 13:00:00,91.0,0.942467,22.537942,28.421796,0
2,B0001-15_1,B0001-15,2023-02-20 13:00:00,91.0,0.942467,5.711927,19.964336,0
3,B0001-22_1,B0001-22,2023-02-20 13:00:00,94.0,0.946876,26.605883,13.039390,1
4,B0001-23_1,B0001-23,2023-02-20 13:00:00,94.0,0.946876,24.037401,35.187848,0
...,...,...,...,...,...,...,...,...
95779,B0729-13_1,B0729-13,2023-02-28 02:00:00,209.0,0.990000,21.438768,21.961071,0
95780,B0729-15_1,B0729-15,2023-02-28 01:00:00,950.0,0.990000,4.248081,4.510619,0
95781,B0729-15_1,B0729-15,2023-02-28 02:00:00,209.0,0.990000,4.248081,6.097206,0
95782,B0729-22_1,B0729-22,2023-02-28 01:00:00,950.0,1.000000,14.851790,20.272872,0


,ID,NE ID,endTime,fault_duration,relation,data_rate_t-1
0,B0017-25_24,B0017-25,2023-02-20 09:00:00,301.0,0.654162,1.175289
1,B0017-25_25,B0017-25,2023-02-20 21:00:00,145.0,0.654162,0.966620
2,B0017-25_26,B0017-25,2023-02-21 00:00:00,250.0,0.654162,1.561278
3,B0017-25_27,B0017-25,2023-02-21 06:00:00,1971.0,0.654162,10.963935
4,B0017-32_1,B0017-32,2023-02-14 21:00:00,3461.0,0.654162,4.471131
...,...,...,...,...,...,...
1927,B0122-43_1,B0122-43,2023-02-11 15:00:00,616.0,1.000000,16.973420
1928,B0122-43_2,B0122-43,2023-02-13 15:00:00,97.0,0.990000,8.685347
1929,B0122-45_1,B0122-45,2023-02-11 15:00:00,616.0,1.000000,3.946578
1930,B0122-45_2,B0122-45,2023-02-13 15:00:00,97.0,0.990000,2.349000


In [7]:
%%time
df_total = df_combined.sort_values(['NE ID','endTime'],ascending=True,ignore_index=True)
del df_total['data_rate_t-1']
df_total_resample = df_total.set_index('endTime').groupby('NE ID').resample('H').asfreq().drop(['NE ID'], 1).reset_index()
df_total_resample = df_total_resample.sort_values(['NE ID','endTime'],ascending=True,ignore_index=True)
print(df_total.shape, df_total_resample.shape)

(1084072, 15) (1093242, 15)
CPU times: user 5.75 s, sys: 263 ms, total: 6.02 s
Wall time: 6.02 s


In [8]:
# df_total['access_success_rate_by_resource_utilition_rate'] = df_total['access_success_rate'] / df_total['resource_utilition_rate']
# df_total_resample['access_success_rate_by_resource_utilition_rate'] = df_total_resample['access_success_rate'] / df_total_resample['resource_utilition_rate']


In [9]:
# df_total = df_total[df_total['data_rate']!=0].reset_index(drop=True)
all_shits = [1,2,3] #list(np.arange(1,49)) # 
for shift_i in tqdm(all_shits):
    for col in ['endTime','access_success_rate','resource_utilition_rate','TA','bler','cqi','mcs','data_rate',
               'fault_duration','relation']:
        df_total[f'{col}_T-{shift_i}'] = df_total.groupby(['NE ID'])[col].shift(shift_i)
    
for shift_i in tqdm(all_shits):
    for col in ['endTime','access_success_rate','resource_utilition_rate','TA','bler','cqi','mcs','data_rate',
               'fault_duration','relation']:
        df_total[f'{col}_T+{shift_i}'] = df_total.groupby(['NE ID'])[col].shift(-shift_i)
        

for shift_i in tqdm(all_shits):
    df_total[f'endTime_T+{shift_i}_hours_elapsed'] = (df_total[f'endTime_T+{shift_i}']-df_total['endTime']).dt.total_seconds() / 3600
    df_total[f'endTime_T-{shift_i}_hours_elapsed'] = (df_total[f'endTime_T-{shift_i}']-df_total['endTime']).dt.total_seconds() / 3600
    del df_total[f'endTime_T+{shift_i}']
    del df_total[f'endTime_T-{shift_i}']
    

100%|██████████| 3/3 [00:00<00:00, 36.25it/s]


In [10]:
# df_total[f'data_rate_T-1-T-2'] = df_total.groupby(['NE ID'])['data_rate'].shift(1)- df_total.groupby(['NE ID'])['data_rate'].shift(2)
# df_total[f'data_rate_T-2-T-3'] = df_total.groupby(['NE ID'])['data_rate'].shift(2)- df_total.groupby(['NE ID'])['data_rate'].shift(3)
# df_total[f'data_rate_T-3-T-4'] = df_total.groupby(['NE ID'])['data_rate'].shift(3)- df_total.groupby(['NE ID'])['data_rate'].shift(4)


In [11]:
all_shits = [24,48,72]
for shift_i in tqdm(all_shits):
    for col in ['access_success_rate','resource_utilition_rate','TA','bler',
                'cqi','mcs','data_rate','fault_duration','relation']:
        df_total_resample[f'{col}_T-{shift_i}'] = df_total_resample.groupby(['NE ID'])[col].shift(shift_i)

for shift_i in tqdm(all_shits):
    for col in ['access_success_rate','resource_utilition_rate','TA','bler',
                'cqi','mcs','data_rate','fault_duration','relation']:
        df_total_resample[f'{col}_T+{shift_i}'] = df_total_resample.groupby(['NE ID'])[col].shift(-shift_i)


100%|██████████| 3/3 [00:02<00:00,  1.31it/s]


In [12]:
df_tot_fault_et = df_total_resample[df_total_resample['fault_duration']>0].groupby(['NE ID']).agg({'endTime':lambda x: list(x)}).add_prefix("fault_").reset_index()


In [13]:
# df_total_resample[f'data_rate_T-24-T-25'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(24)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(25)
# df_total_resample[f'data_rate_T-48-T-49'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(48)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(49)
# df_total_resample[f'data_rate_T-72-T-73'] = df_total_resample.groupby(['NE ID'])['data_rate'].shift(72)- df_total_resample.groupby(['NE ID'])['data_rate'].shift(73)





In [14]:
df_total['month'] = df_total['endTime'].dt.month
df_total['day'] = df_total['endTime'].dt.day
df_total['hour'] = df_total['endTime'].dt.hour
df_total['weekday_number'] = df_total['endTime'].dt.weekday
df_total['hour_sin'] = np.sin(2 * np.pi * df_total['hour'] / 24)
df_total['hour_cos'] = np.cos(2 * np.pi * df_total['hour'] / 24)
# df_total['weekday_sin'] = np.sin(2 * np.pi * df_total['weekday_number'] / 7)
# df_total['weekday_cos'] = np.cos(2 * np.pi * df_total['weekday_number'] / 7)
# df_total['day_sin'] = np.sin(2 * np.pi * df_total['day'] / 31)
# df_total['day_cos'] = np.cos(2 * np.pi * df_total['day'] / 31)

# df_total = df_total.groupby(['ID']).first().reset_index()


In [15]:
%%time
df_mean_drates = df_total.groupby(['NE ID','hour']).agg({'data_rate':['mean','std','min','max','skew'],
                                                         'access_success_rate':['mean','std','min','max','skew'],
                                                         'resource_utilition_rate':['mean','std','min','max','skew'],
                                                         'TA':['mean','std','min','max','skew'],
                                                         'bler':['mean','std','min','max','skew'],
                                                         'cqi':['mean','std','min','max','skew'],
                                                         'mcs':['mean','std','min','max','skew']
                                                        })
df_mean_drates.columns = [i[0]+"_"+i[1] for i in df_mean_drates]
df_mean_drates = df_mean_drates.reset_index()
df_total = df_total.merge(df_mean_drates,on=['NE ID','hour'],how='left')

CPU times: user 1min 9s, sys: 420 ms, total: 1min 10s
Wall time: 1min 10s


In [16]:
# category_cols = ['month','day','hour','weekday_number']

In [17]:
df_total['last_10_data_rates'] = np.vstack([np.array(df_total.groupby(['NE ID'])['data_rate'].shift(i).fillna(-1)) for i in range(1,11)]).T.tolist()


In [18]:
df_train = df_total[df_total['data']=='train_fault'].groupby(['ID']).first().reset_index()
df_train.drop(columns=['access_success_rate','resource_utilition_rate',
                      'TA','bler','cqi','mcs','data_rate','data','TARGET_data_rate'],
              inplace=True)
df_train = df_train.merge(df_total_resample[[col for col in df_total_resample if col not in ['ID','access_success_rate',
                                                 'resource_utilition_rate','TA','bler',
                                                 'cqi','mcs','data_rate','data',
                                                 'TARGET_data_rate','fault_duration',
                                                 'relation','TARGET_data_rate_t+1_trend']]],
              on=['NE ID','endTime'], how='left')

# #time since last fault
# df_train = df_train.merge(df_tot_fault_et,on='NE ID',how='left')
# df_train['fault_endTime'] = df_train.apply(lambda x: max([i for i in x['fault_endTime'] if i<x['endTime']]) if len([i for i in x['fault_endTime'] if i<x['endTime']])>0 else np.nan, axis=1)
# df_train['fault_endTime'] = (df_train['endTime']-df_train['fault_endTime']).dt.total_seconds() / 3600

print(df_train.shape)
display(df_train.head(1))

(7256, 162)


,ID,NE ID,endTime,fault_duration,relation,TARGET_data_rate_t+1_trend,access_success_rate_T-1,resource_utilition_rate_T-1,TA_T-1,bler_T-1,cqi_T-1,mcs_T-1,data_rate_T-1,fault_duration_T-1,relation_T-1,access_success_rate_T-2,resource_utilition_rate_T-2,TA_T-2,bler_T-2,cqi_T-2,mcs_T-2,data_rate_T-2,fault_duration_T-2,relation_T-2,access_success_rate_T-3,resource_utilition_rate_T-3,TA_T-3,bler_T-3,cqi_T-3,mcs_T-3,data_rate_T-3,fault_duration_T-3,relation_T-3,access_success_rate_T+1,resource_utilition_rate_T+1,TA_T+1,bler_T+1,cqi_T+1,mcs_T+1,data_rate_T+1,fault_duration_T+1,relation_T+1,access_success_rate_T+2,resource_utilition_rate_T+2,TA_T+2,bler_T+2,cqi_T+2,mcs_T+2,data_rate_T+2,fault_duration_T+2,relation_T+2,access_success_rate_T+3,resource_utilition_rate_T+3,TA_T+3,bler_T+3,cqi_T+3,mcs_T+3,data_rate_T+3,fault_duration_T+3,relation_T+3,endTime_T+1_hours_elapsed,endTime_T-1_hours_elapsed,endTime_T+2_hours_elapsed,endTime_T-2_hours_elapsed,endTime_T+3_hours_elapsed,endTime_T-3_hours_elapsed,month,day,hour,weekday_number,hour_sin,hour_cos,data_rate_mean,data_rate_std,data_rate_min,data_rate_max,data_rate_skew,access_success_rate_mean,access_success_rate_std,access_success_rate_min,access_success_rate_max,access_success_rate_skew,resource_utilition_rate_mean,resource_utilition_rate_std,resource_utilition_rate_min,resource_utilition_rate_max,resource_utilition_rate_skew,TA_mean,TA_std,TA_min,TA_max,TA_skew,bler_mean,bler_std,bler_min,bler_max,bler_skew,cqi_mean,cqi_std,cqi_min,cqi_max,cqi_skew,mcs_mean,mcs_std,mcs_min,mcs_max,mcs_skew,last_10_data_rates,access_success_rate_T-24,resource_utilition_rate_T-24,TA_T-24,bler_T-24,cqi_T-24,mcs_T-24,data_rate_T-24,fault_duration_T-24,relation_T-24,access_success_rate_T-48,resource_utilition_rate_T-48,TA_T-48,bler_T-48,cqi_T-48,mcs_T-48,data_rate_T-48,fault_duration_T-48,relation_T-48,access_success_rate_T-72,resource_utilition_rate_T-72,TA_T-72,bler_T-72,cqi_T-72,mcs_T-72,data_rate_T-72,fault_duration_T-72,relation_T-72,access_success_rate_T+24,resource_utilition_rate_T+24,TA_T+24,bler_T+24,cqi_T+24,mcs_T+24,data_rate_T+24,fault_duration_T+24,relation_T+24,access_success_rate_T+48,resource_utilition_rate_T+48,TA_T+48,bler_T+48,cqi_T+48,mcs_T+48,data_rate_T+48,fault_duration_T+48,relation_T+48,access_success_rate_T+72,resource_utilition_rate_T+72,TA_T+72,bler_T+72,cqi_T+72,mcs_T+72,data_rate_T+72,fault_duration_T+72,relation_T+72
0,B0001-12_1,B0001-12,2023-02-20 13:00:00,91.0,0.942467,0.0,100.0,29.382,0.763943,10.336147,8.241281,11.089316,23.253083,0.0,0.0,100.0,45.419,0.811247,10.261306,8.565688,12.344837,21.86461,0.0,0.0,100.0,45.113,0.75943,10.249399,8.600942,12.360695,24.524838,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,-2.0,NaN,-3.0,2,20,13,0,-0.258819,-0.965926,24.222605,5.65253,17.156717,32.484436,0.150417,99.974572,0.03459,99.892357,100.0,-1.427632,27.788091,8.72487,18.661,44.245,0.847005,0.855253,0.029898,0.796883,0.896579,-0.401494,10.89613,0.750986,10.004353,12.333499,0.541658,8.281904,0.157822,8.071295,8.566765,0.260891,11.131713,1.224448,9.865972,14.055772,1.46403,"[23.25308286587564, 21.864609567544047, 24.524...",100.0,25.604,0.891304,10.965842,8.320807,11.496593,17.156717,0.0,0.0,100.0,37.071,0.862605,11.659417,8.081238,10.35328,17.371207,0.0,0.0,99.951714,31.598,0.834253,10.487901,8.34011,9.946214,22.470461,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_test = df_total[df_total['data']=='test_fault'].reset_index(drop=True)
df_test.drop(columns=['access_success_rate','resource_utilition_rate',
                      'TA','bler','cqi','mcs','data_rate','data',
                      'TARGET_data_rate','TARGET_data_rate_t+1_trend'],
             inplace=True)
df_test = df_test.merge(df_total_resample[[col for col in df_total_resample if col not in ['ID','access_success_rate',
                                                 'resource_utilition_rate','TA','bler',
                                                 'cqi','mcs','data_rate','data',
                                                 'TARGET_data_rate','fault_duration',
                                                 'relation','TARGET_data_rate_t+1_trend']]],
              on=['NE ID','endTime'], how='left')

# #time since last fault
# df_test = df_test.merge(df_tot_fault_et,on='NE ID',how='left')
# df_test['fault_endTime'] = df_test.apply(lambda x: max([i for i in x['fault_endTime'] if i<x['endTime']]) if len([i for i in x['fault_endTime'] if i<x['endTime']])>0 else np.nan, axis=1)
# df_test['fault_endTime'] = (df_test['endTime']-df_test['fault_endTime']).dt.total_seconds() / 3600


print(df_test.shape)
display(df_test.head(1))

(1932, 161)


,ID,NE ID,endTime,fault_duration,relation,access_success_rate_T-1,resource_utilition_rate_T-1,TA_T-1,bler_T-1,cqi_T-1,mcs_T-1,data_rate_T-1,fault_duration_T-1,relation_T-1,access_success_rate_T-2,resource_utilition_rate_T-2,TA_T-2,bler_T-2,cqi_T-2,mcs_T-2,data_rate_T-2,fault_duration_T-2,relation_T-2,access_success_rate_T-3,resource_utilition_rate_T-3,TA_T-3,bler_T-3,cqi_T-3,mcs_T-3,data_rate_T-3,fault_duration_T-3,relation_T-3,access_success_rate_T+1,resource_utilition_rate_T+1,TA_T+1,bler_T+1,cqi_T+1,mcs_T+1,data_rate_T+1,fault_duration_T+1,relation_T+1,access_success_rate_T+2,resource_utilition_rate_T+2,TA_T+2,bler_T+2,cqi_T+2,mcs_T+2,data_rate_T+2,fault_duration_T+2,relation_T+2,access_success_rate_T+3,resource_utilition_rate_T+3,TA_T+3,bler_T+3,cqi_T+3,mcs_T+3,data_rate_T+3,fault_duration_T+3,relation_T+3,endTime_T+1_hours_elapsed,endTime_T-1_hours_elapsed,endTime_T+2_hours_elapsed,endTime_T-2_hours_elapsed,endTime_T+3_hours_elapsed,endTime_T-3_hours_elapsed,month,day,hour,weekday_number,hour_sin,hour_cos,data_rate_mean,data_rate_std,data_rate_min,data_rate_max,data_rate_skew,access_success_rate_mean,access_success_rate_std,access_success_rate_min,access_success_rate_max,access_success_rate_skew,resource_utilition_rate_mean,resource_utilition_rate_std,resource_utilition_rate_min,resource_utilition_rate_max,resource_utilition_rate_skew,TA_mean,TA_std,TA_min,TA_max,TA_skew,bler_mean,bler_std,bler_min,bler_max,bler_skew,cqi_mean,cqi_std,cqi_min,cqi_max,cqi_skew,mcs_mean,mcs_std,mcs_min,mcs_max,mcs_skew,last_10_data_rates,access_success_rate_T-24,resource_utilition_rate_T-24,TA_T-24,bler_T-24,cqi_T-24,mcs_T-24,data_rate_T-24,fault_duration_T-24,relation_T-24,access_success_rate_T-48,resource_utilition_rate_T-48,TA_T-48,bler_T-48,cqi_T-48,mcs_T-48,data_rate_T-48,fault_duration_T-48,relation_T-48,access_success_rate_T-72,resource_utilition_rate_T-72,TA_T-72,bler_T-72,cqi_T-72,mcs_T-72,data_rate_T-72,fault_duration_T-72,relation_T-72,access_success_rate_T+24,resource_utilition_rate_T+24,TA_T+24,bler_T+24,cqi_T+24,mcs_T+24,data_rate_T+24,fault_duration_T+24,relation_T+24,access_success_rate_T+48,resource_utilition_rate_T+48,TA_T+48,bler_T+48,cqi_T+48,mcs_T+48,data_rate_T+48,fault_duration_T+48,relation_T+48,access_success_rate_T+72,resource_utilition_rate_T+72,TA_T+72,bler_T+72,cqi_T+72,mcs_T+72,data_rate_T+72,fault_duration_T+72,relation_T+72
0,B0017-25_24,B0017-25,2023-02-20 09:00:00,301.0,0.654162,99.357688,84.004,2.923368,14.209819,5.582824,5.667775,1.175289,0.0,0.0,99.927484,24.264,3.145402,10.015796,6.141206,6.544645,6.777114,0.0,0.0,99.714558,8.678,3.71541,7.653169,5.964988,5.403557,6.815789,0.0,0.0,99.264875,93.086,2.732594,16.947693,5.475508,4.719393,0.52254,0.0,0.0,99.642289,92.242,2.877206,15.083843,5.628569,5.051611,0.96662,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,0.654162,10.0,-1.0,11.0,-2.0,12.0,-3.0,2,20,9,0,0.707107,-0.707107,1.952016,1.752728,0.487686,5.434899,1.230667,99.447719,0.373911,98.659517,99.814929,-1.478284,77.22725,21.93246,37.79,95.126,-0.930093,2.944483,0.193886,2.785595,3.290713,1.058453,14.291685,1.744582,11.698591,16.615224,-0.170393,5.675014,0.219699,5.293939,5.909254,-0.583507,5.815168,0.821974,4.738793,6.893467,0.206865,"[1.17528900605546, 6.777114273500261, 6.815788...",99.814929,55.07,3.171086,12.769518,5.834403,6.893467,3.292267,0.0,0.0,99.754199,78.256,2.889007,13.597193,5.825114,6.012004,2.046892,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.0,0.654162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
remove_non_unique_cols = []
for col in df_train:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend','last_10_data_rates']:
        if df_train[col].nunique()<=1:
            remove_non_unique_cols.append(col)
            print(col,":",df_train[col].nunique())

for col in df_test:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend','last_10_data_rates']:
        if df_test[col].nunique()<=1:
            remove_non_unique_cols.append(col)
            print(col,":",df_test[col].nunique())
remove_non_unique_cols

fault_duration_T-1 : 1
relation_T-1 : 1
fault_duration_T+1 : 1
relation_T+1 : 1


['fault_duration_T-1', 'relation_T-1', 'fault_duration_T+1', 'relation_T+1']

In [21]:
feature_set = [col for col in df_test.columns if col not in ['ID','NE ID','endTime','last_10_data_rates']]
print(len(feature_set))
feature_set = [col for col in feature_set if col not in remove_non_unique_cols]
print(len(feature_set))
target_variable = 'TARGET_data_rate_t+1_trend'
id_variable = 'ID'

157
153


In [22]:
for col in df_test:
    if col not in ['TARGET_data_rate','TARGET_data_rate_t+1_trend']:
        if df_test[col].isnull().sum()/len(df_test)>=0.8:
            print(col,":",df_test[col].isnull().sum()/len(df_test))

In [23]:
train_x = df_train.reset_index(drop=True)
train_y = train_x[target_variable]
test_x = df_test.reset_index(drop=True)

del train_x[target_variable]
test_id = test_x[id_variable]
# train_x = train_x[feature_set].copy()
# test_x = test_x[feature_set].copy()

print(train_x.shape, train_y.shape, test_x.shape)

(7256, 161) (7256,) (1932, 161)


In [24]:
def evaluate_F1_lgb(truth, predictions):  
    predictions = np.where(predictions < f1_threshold, 0, 1)
    f1 = f1_score(truth, predictions)
    return ('F1 Score', f1, True)

def evaluate_F1_xgb(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = f1_score(y_true, np.round(y_pred))
    return 'f1_err', -err

NFOLDS = 10
sc_pos_wt = 1.5
f1_threshold=0.5
seeds_rs = [42] #,1818,181812,1995,1212

In [25]:
#0.7379
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_lgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = LGBMClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                               n_jobs=-1,lambda_l1=1,feature_fraction=0.45,
#                                lambda_l2=1,
                               scale_pos_weight=sc_pos_wt,
#                                max_depth=8,
#                                linear_tree=True,linear_lambda=0.1
                              )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
#                   categorical_feature=['endTime'],
                  eval_metric=evaluate_F1_lgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_lgb = pd.concat([df_train_error_analysis_lgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_lgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"lgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"lgb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_lgb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
FOLD F1 Score: 0.758703481392557


● Fold : 2
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
FOLD F1 Score: 0.7450509898020395


● Fold : 3
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.45
FOLD F1 Score: 0.7391646966115051


● Fold : 4
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] feature_fraction is set=0.45, colsample_bytree=1.0 wil

,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.667235,0.642579,0.521445,0.520145,0.541610,0.511655,0.653556,0.536481,0.693960,0.559897,B0017-25_24,0.577462
1,0.359288,0.431417,0.470529,0.446514,0.362042,0.492654,0.360341,0.425419,0.390719,0.385074,B0017-25_25,0.407497
2,0.279218,0.273019,0.453207,0.456731,0.270626,0.471672,0.215169,0.376926,0.178245,0.286746,B0017-25_26,0.295390
3,0.805695,0.791328,0.565793,0.586728,0.805961,0.531703,0.763487,0.660961,0.822354,0.841509,B0017-25_27,0.698068
4,0.766233,0.851015,0.532060,0.572211,0.840458,0.533762,0.807354,0.642813,0.845350,0.782595,B0017-32_1,0.693213


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,data_rate_T-1,325.5
1,data_rate_mean,146.1
2,data_rate_min,56.6
3,data_rate_T+1,54.6
4,data_rate_T+2,42.0
...,...,...
148,endTime_T+1_hours_elapsed,0.3
149,endTime_T-2_hours_elapsed,0.1
150,endTime_T-3_hours_elapsed,0.1
151,endTime_T-1_hours_elapsed,0.0


,feature,importance
0,data_rate_T-1,325.5
1,data_rate_mean,146.1
2,data_rate_min,56.6
3,data_rate_T+1,54.6
4,data_rate_T+2,42.0
5,data_rate_T-72,37.7
6,data_rate_T-3,36.9
7,data_rate_max,32.1
8,data_rate_T-2,31.1
9,hour,28.8


In [26]:
#0.7314
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_cb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = CatBoostClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                                   eval_metric='F1:use_weights=False',
                                   scale_pos_weight=sc_pos_wt,
                                   use_best_model=True,
                                   max_depth=8
                                  )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
                  use_best_model=True,
    #               cat_features=category_cols
    #               eval_metric=evaluate_F1_lgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    

    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_cb = pd.concat([df_train_error_analysis_cb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_cb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"cb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"cb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_cb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
FOLD F1 Score: 0.7561837455830389


● Fold : 2
FOLD F1 Score: 0.7373219373219374


● Fold : 3
FOLD F1 Score: 0.7319819819819819


● Fold : 4
FOLD F1 Score: 0.7306190741773564


● Fold : 5
FOLD F1 Score: 0.7300380228136882


● Fold : 6
FOLD F1 Score: 0.7338345864661653


● Fold : 7
FOLD F1 Score: 0.7318489835430785


● Fold : 8
FOLD F1 Score: 0.7327769132833499


● Fold : 9
FOLD F1 Score: 0.7303805364940735


● Fold : 10
FOLD F1 Score: 0.731486715658564


Final F1 Score: 0.731486715658564
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.723677,0.677687,0.650506,0.559009,0.697297,0.769205,0.736467,0.742491,0.617116,0.730971,B0017-25_24,0.684422
1,0.484526,0.428557,0.501693,0.443254,0.493011,0.453397,0.462249,0.503764,0.526203,0.452672,B0017-25_25,0.473073
2,0.198508,0.283390,0.265510,0.349072,0.294846,0.203088,0.308182,0.261959,0.377843,0.174684,B0017-25_26,0.256763
3,0.812679,0.744491,0.728276,0.702091,0.772766,0.820644,0.730610,0.751878,0.721686,0.819554,B0017-25_27,0.758250
4,0.679760,0.755601,0.706514,0.612391,0.738236,0.864245,0.755628,0.708732,0.647912,0.754298,B0017-32_1,0.716430


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,data_rate_T-1,20.053837
1,data_rate_mean,5.100866
2,hour,3.462352
3,data_rate_min,3.327797
4,hour_cos,2.476949
...,...,...
148,endTime_T+3_hours_elapsed,0.034826
149,endTime_T+2_hours_elapsed,0.021466
150,endTime_T-2_hours_elapsed,0.018111
151,endTime_T-1_hours_elapsed,0.011176


,feature,importance
0,data_rate_T-1,20.053837
1,data_rate_mean,5.100866
2,hour,3.462352
3,data_rate_min,3.327797
4,hour_cos,2.476949
5,TA_T-1,2.276596
6,data_rate_T+1,1.676379
7,cqi_T-1,1.570848
8,data_rate_T+2,1.427299
9,mcs_T-1,1.408397


In [27]:
#0.7248
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_hgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = HistGradientBoostingClassifier(random_state=rs_,verbose=0,
                                               scoring = 'recall',
                                              early_stopping = True,
                                               max_iter = 10000,
                                              learning_rate = 0.03,
                                              n_iter_no_change = 100,
                                               class_weight={1:sc_pos_wt},
                                               )
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids])
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_hgb = pd.concat([df_train_error_analysis_hgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
#     fold_importance_df["importance"] = model.feat
#     fold_importance_df["fold"] = j + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#     j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_hgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"hgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"hgb_{version_nb}.csv", index = False)
display(df_submission.head())

# all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
# all_features.reset_index(inplace=True)
# important_features = list(all_features[all_features['importance']!=0]['feature'])
# display(all_features)
# display(all_features.head(25))

final_prediction_hgb = final_prediction[['ID','data_rate_t+1_trend']].copy()


● Fold : 1
FOLD F1 Score: 0.7372262773722628


● Fold : 2
FOLD F1 Score: 0.7317661241711874


● Fold : 3
FOLD F1 Score: 0.7239312824610468


● Fold : 4
FOLD F1 Score: 0.7197586726998493


● Fold : 5
FOLD F1 Score: 0.7205029013539652


● Fold : 6
FOLD F1 Score: 0.7296051312888354


● Fold : 7
FOLD F1 Score: 0.7256789274664833


● Fold : 8
FOLD F1 Score: 0.7238986119493059


● Fold : 9
FOLD F1 Score: 0.7241885935621744


● Fold : 10
FOLD F1 Score: 0.724846736386585


Final F1 Score: 0.724846736386585
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.574754,0.676968,0.713827,0.577579,0.638053,0.704223,0.706723,0.652426,0.484550,0.586377,B0017-25_24,0.622918
1,0.388435,0.411181,0.397059,0.408820,0.394316,0.507618,0.424612,0.460020,0.267113,0.355106,B0017-25_25,0.391148
2,0.308357,0.291582,0.230204,0.322949,0.199135,0.237786,0.346266,0.268190,0.214371,0.256836,B0017-25_26,0.259638
3,0.824171,0.770264,0.790095,0.817815,0.818824,0.812425,0.811038,0.761406,0.779845,0.818595,B0017-25_27,0.799844
4,0.799099,0.825246,0.840636,0.757439,0.831779,0.850831,0.833545,0.801242,0.854807,0.787604,B0017-32_1,0.817138


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


In [28]:
# train_x[category_cols] = train_x[category_cols].astype("category")
# test_x[category_cols] = test_x[category_cols].astype("category")
#  enable_categorical=True,

In [29]:
# 0.7344
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
pred_test = []
pred_x = []
pred_y = []
j=0
df_train_error_analysis_xgb = pd.DataFrame()
feature_importance_df = pd.DataFrame()
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    pred_fold = []
    pred_fold_test = []
    for rs_ in seeds_rs:
        model = XGBClassifier(n_estimators=10000, random_state=rs_, learning_rate=0.03,
                              n_jobs=-1,scale_pos_weight=sc_pos_wt,colsample_bytree=0.45,
                              reg_lambda=1,reg_alpha=0,max_depth=7,
                             disable_default_eval_metric=1,tree_method='exact')
        model.fit(train_x.loc[train_ids][feature_set], train_y.loc[train_ids], 
                  eval_set=[(train_x.loc[train_ids][feature_set], train_y.loc[train_ids]), 
                            (train_x.loc[test_ids][feature_set], train_y.loc[test_ids])],
                  verbose=False,
                  early_stopping_rounds=100,
                  eval_metric=evaluate_F1_xgb
                 )
        pred_fold_i = model.predict_proba(train_x.loc[test_ids][feature_set])[:,-1]
        pred_fold.append(pred_fold_i)
        pred_fold_test_i = model.predict_proba(test_x[feature_set])[:,-1]
        pred_fold_test.append(pred_fold_test_i)
    pred_fold = [hmean(i) for i in [list(i) for i in zip(*pred_fold)]]
    pred_fold_test = [hmean(i) for i in [list(i) for i in zip(*pred_fold_test)]]
    
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    
    pred_test.append([float(i) for i  in pred_fold_test])
    print("FOLD F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < f1_threshold, 0, 1))))
    df_train_error_analysis_fold = pd.DataFrame({'ID':list(train_x.loc[test_ids]['ID']),
                                        'ACTUAL_TARGET':list(train_y.loc[test_ids]),
                                        'PREDICTED_TARGET':pred_fold})
    df_train_error_analysis_xgb = pd.concat([df_train_error_analysis_xgb,df_train_error_analysis_fold],ignore_index=True)
    print('\n')

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids][feature_set].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

# for i_thr in [np.round(i,2) for i in np.arange(0.2,0.6,0.01)]:
#     print("Threshold:",i_thr,"Final F1 Score:", f1_score(pred_y, list(np.where(np.array(pred_x) < i_thr, 0, 1))))
pred_x = list(np.where(np.array(pred_x) < f1_threshold, 0, 1))
print("Final F1 Score:", f1_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

# vote = final_prediction.mean(axis=1)
vote = final_prediction.apply(lambda x: hmean(x), axis=1)
final_prediction[id_variable] = test_id.values
final_prediction['data_rate_t+1_trend'] = vote
print(final_prediction.shape)

train_pred_all = pd.DataFrame()
train_pred_all['data_rate_t+1_trend'] = pred_y
train_pred_all[f'model_{version_nb}'] = pred_x
final_prediction_xgb = final_prediction.copy()

display(final_prediction.head())
df_submission = final_prediction[[id_variable,'data_rate_t+1_trend']].copy()
df_submission.to_csv(f"xgb_{version_nb}_raw.csv", index = False)
df_submission['data_rate_t+1_trend'] = list(np.where(final_prediction['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_submission.to_csv(f"xgb_{version_nb}.csv", index = False)
display(df_submission.head())

all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
display(all_features)
display(all_features.head(25))

final_prediction_xgb = final_prediction[['ID','data_rate_t+1_trend']].copy()



● Fold : 1
FOLD F1 Score: 0.7476190476190476


● Fold : 2
FOLD F1 Score: 0.7331042382588774


● Fold : 3
FOLD F1 Score: 0.7332826747720365


● Fold : 4
FOLD F1 Score: 0.7326789838337183


● Fold : 5
FOLD F1 Score: 0.7335526315789476


● Fold : 6
FOLD F1 Score: 0.7366167023554604


● Fold : 7
FOLD F1 Score: 0.7356820838203373


● Fold : 8
FOLD F1 Score: 0.735052754982415


● Fold : 9
FOLD F1 Score: 0.7338447536081134


● Fold : 10
FOLD F1 Score: 0.7344262295081968


Final F1 Score: 0.7344262295081968
(1932, 12)


,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,ID,data_rate_t+1_trend
0,0.592260,0.642446,0.623138,0.556926,0.635579,0.734785,0.632818,0.757178,0.673710,0.696653,B0017-25_24,0.649336
1,0.219484,0.395786,0.345545,0.157910,0.159701,0.370816,0.337705,0.226059,0.268052,0.250092,B0017-25_25,0.247636
2,0.239681,0.355374,0.248112,0.252844,0.100433,0.302964,0.309252,0.112823,0.229123,0.181634,B0017-25_26,0.199218
3,0.798538,0.676806,0.729351,0.815093,0.835220,0.738669,0.752295,0.738571,0.781497,0.771277,B0017-25_27,0.761139
4,0.702278,0.670795,0.762262,0.728774,0.834196,0.790215,0.787644,0.829670,0.877409,0.754409,B0017-32_1,0.769044


,ID,data_rate_t+1_trend
0,B0017-25_24,1
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,1
4,B0017-32_1,1


,feature,importance
0,data_rate_T-1,0.021071
1,hour,0.020889
2,cqi_T-1,0.019775
3,data_rate_mean,0.017162
4,TA_T-1,0.013496
...,...,...
148,day,0.004495
149,endTime_T-3_hours_elapsed,0.004191
150,endTime_T+1_hours_elapsed,0.004110
151,endTime_T-2_hours_elapsed,0.003696


,feature,importance
0,data_rate_T-1,0.021071
1,hour,0.020889
2,cqi_T-1,0.019775
3,data_rate_mean,0.017162
4,TA_T-1,0.013496
5,data_rate_T+1,0.013471
6,data_rate_min,0.012591
7,data_rate_max,0.012056
8,resource_utilition_rate_T-1,0.011789
9,resource_utilition_rate_max,0.011439


In [30]:
for df in [final_prediction_lgb,final_prediction_cb,final_prediction_hgb,final_prediction_xgb]:
    print(df['data_rate_t+1_trend'].min(),df['data_rate_t+1_trend'].max())

0.0844364125096702 0.7563178536452758
0.05443040563925646 0.9340851164061182
0.03154348346206394 0.9122118449286292
0.05420787732721628 0.8816353861533244


In [31]:
#0.7330
df_local_cv = df_train_error_analysis_lgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_LGB").join(df_train_error_analysis_xgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_XGB")).join(df_train_error_analysis_cb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_CB")).join(df_train_error_analysis_hgb.set_index(["ID","ACTUAL_TARGET"]).add_suffix("_HGB"))
# vote = df_local_cv['PREDICTED_TARGET_LGB']*0.4+df_local_cv['PREDICTED_TARGET_XGB']*0.4+df_local_cv['PREDICTED_TARGET_HGB']*0.1+df_local_cv['PREDICTED_TARGET_CB']*0.1
vote = df_local_cv.apply(lambda x: hmean(x), axis=1)
df_local_cv['data_rate_t+1_trend'] = vote
df_local_cv['data_rate_t+1_trend'] = list(np.where(df_local_cv['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_local_cv = df_local_cv.reset_index()
f1_score(df_local_cv['ACTUAL_TARGET'],df_local_cv['data_rate_t+1_trend'])

0.7330733782346686

In [32]:
df_ensemble = final_prediction_lgb.set_index("ID").add_suffix("_LGB").join(final_prediction_cb.set_index("ID").add_suffix("_CB")).join(final_prediction_hgb.set_index("ID").add_suffix("_HGB")).join(final_prediction_xgb.set_index("ID").add_suffix("_XGB"))
display(df_ensemble.corr())
vote = df_ensemble.apply(lambda x: hmean(x), axis=1)
# vote = df_ensemble['data_rate_t+1_trend_LGB']*0.4+df_ensemble['data_rate_t+1_trend_XGB']*0.4+df_ensemble['data_rate_t+1_trend_HGB']*0.1+df_ensemble['data_rate_t+1_trend_CB']*0.1
df_ensemble['data_rate_t+1_trend'] = vote
df_ensemble = df_ensemble[['data_rate_t+1_trend']].copy()
df_ensemble_raw = df_ensemble.reset_index()
df_ensemble_raw.to_csv(f"ensemble_all_{version_nb}_{sc_pos_wt}_raw.csv", index = False)
df_ensemble['data_rate_t+1_trend'] = list(np.where(df_ensemble['data_rate_t+1_trend'] < f1_threshold, 0, 1))
df_ensemble = df_ensemble.reset_index()
df_ensemble.to_csv(f"ensemble_all_{version_nb}_{sc_pos_wt}.csv", index = False)


,data_rate_t+1_trend_LGB,data_rate_t+1_trend_CB,data_rate_t+1_trend_HGB,data_rate_t+1_trend_XGB
data_rate_t+1_trend_LGB,1.000000,0.950761,0.975619,0.985080
data_rate_t+1_trend_CB,0.950761,1.000000,0.928131,0.952036
data_rate_t+1_trend_HGB,0.975619,0.928131,1.000000,0.965881
data_rate_t+1_trend_XGB,0.985080,0.952036,0.965881,1.000000


In [33]:
df_ensemble['data_rate_t+1_trend'].value_counts(normalize=True)

1    0.694099
0    0.305901
Name: data_rate_t+1_trend, dtype: float64